In [2]:
import sqlite3

# all the verbs, ordered
QUERY = '''
  select a.verb from appdb a
  ORDER BY a.freq DESC
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
verbs = [row[0] for row in c.execute(QUERY)]
len(verbs), verbs[:5], verbs[-5:]

(1974,
 ['andare', 'aprire', 'avere', 'bere', 'cadere'],
 ['bluffare', 'spettegolare', 'fraternizzare', 'pomiciare', 'internalizzare'])

In [3]:
# just the irregular verbs
QUERY = '''
  select a.verb from appdb a
  WHERE NOT a.irregular
  ORDER BY a.freq DESC  
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
regulars = [row for row in c.execute(QUERY)]
regulars[0:50]

[('farsi',),
 ('darsi',),
 ('trovare',),
 ('trovarsi',),
 ('vedere',),
 ('vedersi',),
 ('mettere',),
 ('mettersi',),
 ('scrivere',),
 ('scriversi',),
 ('presentare',),
 ('presentarsi',),
 ('parlare',),
 ('pensare',),
 ('prendere',),
 ('chiedere',),
 ('chiedersi',),
 ('tenere',),
 ('portare',),
 ('rendere',),
 ('rendersi',),
 ('sembrare',),
 ('trattare',),
 ('cercare',),
 ('arrivare',),
 ('sentire',),
 ('sentirsi',),
 ('svolgere',),
 ('credere',),
 ('inviare',),
 ('ricordare',),
 ('ricordarsi',),
 ('riguardare',),
 ('lasciare',),
 ('considerare',),
 ('parere',),
 ('utilizzare',),
 ('passare',),
 ('ritenere',),
 ('seguire',),
 ('diventare',),
 ('leggere',),
 ('leggersi',),
 ('vivere',),
 ('conoscersi',),
 ('realizzare',),
 ('usare',),
 ('consentire',),
 ('tornare',),
 ('capire',)]

In [4]:
# just the irregular verbs
QUERY = '''
  select a.verb from appdb a
  WHERE a.irregular
  ORDER BY a.freq DESC  
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
irregulars = [row[0] for row in c.execute(QUERY)]
irregulars

['andare',
 'aprire',
 'avere',
 'bere',
 'cadere',
 'conoscere',
 'dare',
 'dire',
 'dovere',
 'essere',
 'fare',
 'potere',
 'salire',
 'sapere',
 'stare',
 'tradurre',
 'uscire',
 'venire',
 'volere']

In [ ]:
# just the irregular verbs
QUERY = '''
  select a.verb from appdb a
  WHERE a.irregular
  ORDER BY a.freq DESC  
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
irregulars = [row[0] for row in c.execute(QUERY)]

In [61]:
if 2: print('hello')

hello


In [63]:
import random
from collections import defaultdict

def genderizer(pos):
    if pos == 2: return(random.choice([2, 3]))
    if pos == 6: return(random.choice([6, 7]))
    return(pos)   

def pick_pos(k):
    options = [0,1,2,4,5,6]
    choices = random.sample(options, k=k)
    return([genderizer(choice) for choice in choices])

for i in range(10):
    print(sorted(pick_pos(3)))

[1, 4, 5]
[0, 1, 5]
[1, 3, 6]
[2, 4, 5]
[1, 5, 7]
[1, 2, 4]
[0, 2, 6]
[0, 2, 6]
[0, 4, 7]
[0, 4, 6]


In [47]:
# query a single verb
QUERY = '''
  select * FROM cards c 
  WHERE c.verb = '%s'
'''
conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
forms = [row for row in c.execute(QUERY % 'dare')]
len(forms)

112

In [ ]:
# A1 deck
QUERY = '''
  select a.*, c.ROWID, c.pos, c.verb, c.form, c.conjugation, f.level from cards c
  INNER JOIN appdb a ON c.verb = a.verb 
  INNER JOIN forms f ON f.form = c.form
  WHERE c.form = 'INDICATIVO_PASSATO_PROSSIMO' AND a.irregular
  ORDER BY a.freq DESC
  LIMIT 1
  '''

QUERY = '''
  select a.*, c.ROWID, c.pos, c.verb, c.form, c.conjugation, f.level from cards c
  INNER JOIN appdb a ON c.verb = a.verb 
  INNER JOIN forms f ON f.form = c.form
  WHERE c.form = 'INDICATIVO_PASSATO_PROSSIMO' AND a.irregular
  ORDER BY a.freq DESC
  '''

conn = sqlite3.connect('/Users/davidp/work/verbe-italiani/data/db/app.db')
c = conn.cursor()
results = [row for row in c.execute(QUERY)]
results

In [ ]:
import urllib
url = 'https://conjugator.reverso.net/index-italian-1-250.html'

def request_contents(url):
    try:
        contents = urllib.request.urlopen(url)
    except urllib.error.HTTPError:
        return (None)
    return (contents.read().decode(encoding="utf-8", errors="strict"))

In [ ]:
html = request_contents(url)

In [ ]:
from cmd import Cmd

def show_card():
    print('''
    Presente: io canto
    
    futuro: io canterò
    ''')

class MyPrompt(Cmd):
    def __init__(self):
        super().__init__()
        self.counter = 12

    def do_increment(self, input):
        self.counter += 1
        
    def do_0(self, input):
        self.counter = 0
    
    def do_card(self, input):
        show_card()
    
    def do_exit(self, inp):
        print("Bye", self.counter)
        return True
 
    def do_add(self, inp):
        print("Adding '{}'".format(inp))
 
MyPrompt().cmdloop()

In [ ]:
import csv
from collections import defaultdict
import random
import os

url1 = '/Users/davidp/work/verbe-italiani/data/csvs/top236.csv'

forms = [
    'Condizionale Passato',
    'Condizionale Presente',
    'Congiuntivo Imperfetto',
    'Congiuntivo Perfetto',
    'Congiuntivo Presente',
    'Congiuntivo Trapassato',
    'Futuro',
    'Futuro Anteriore',
    'Imperfetto',
    'Passato Remoto',
    'Perfetto',
#    'Presente',
    'Trapassato Prossimo'
]

In [ ]:
random.choice(range(8))

In [ ]:
def write_full_csv(url, cards):
    with open(url, encoding='utf-8', mode='w') as f:
        w = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for card in cards:
            w.writerow(card)

In [ ]:
def read_full_csv(url):
    db = defaultdict(dict)
    with open(url, encoding='utf-8', mode='r') as f:
        r = csv.reader(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
        for i, row in enumerate(r):
            if i == 0:
                continue
            _, pos, root, form, front, back = row
            if not form in db[root].keys():
                db[root][form] = ['' for i in range(8)]            
            db[root][form][int(pos)] = back
    return(db)

In [ ]:
x = 'Abc'
'-'.join(x.lower().split(' '))

In [ ]:
def one_form(db, form):
    keys = list(db.keys())
    random.shuffle(keys)
    
    cards = []
    imperfetto = []
    for root in db.keys():
        presente = db[root]['Presente']
        i = random.choice(range(8))
        front = '%s --> %s' %(presente[i], form)
        back = db[root][form][i]
        cards += [[front, back]]
    return(cards)
    
def main(url):
    db = read_full_csv(url1)
    for form in forms:
        cards = one_form(db, form)
        base, ext = os.path.splitext(url)
        suffix = '-'.join(form.lower().split(' '))
        output = '%s-%s%s' %(base, suffix, ext)
        write_full_csv(output, cards)

main(url1)